# particle_interp_compare

compare particle simulations in different fieldsets with a single particle

In [ ]:
%matplotlib inline

In [ ]:
from datetime import timedelta

import cartopy
import cartopy.crs as ccrs
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, plotTrajectoriesFile

from utils import create_path
from parcels_utils import get_file_info
from plot_utils import plot_trajectories

In [ ]:
dir1km = "west_coast_1km_hourly_netcdfs/"

files = [
    get_file_info(dir1km + "west_coast_1km_hourly_region_tijuana_river_lin_aggr.nc", 1, name="lin_aggr"),
    get_file_info(dir1km + "west_coast_1km_hourly_region_tijuana_river_lin.nc", 1, name="lin"),
    get_file_info(dir1km + "west_coast_1km_hourly_region_tijuana_river_simpterp.nc", 1, name="simp"),
    get_file_info(dir1km + "west_coast_1km_hourly_region_tijuana_river_invdist.nc", 1, name="invd")
]

In [ ]:
(files[0]["xrds"]["time"].max().values - np.datetime64("2020-06-17T09:00:00")) / np.timedelta64(1, 's')

In [ ]:
(np.datetime64("2020-06-17T09:00:00") - files[0]["xrds"]["time"].min().values) / np.timedelta64(1, 's')

In [ ]:
start_time = files[0]["xrds"]["time"].min().values
end_time = files[0]["xrds"]["time"].max().values
spawn_time = np.datetime64("2020-06-17T09:00:00")
seconds_delta = (spawn_time - start_time) / np.timedelta64(1, "s")
spawn_loc = (32.551707, -117.138)  # (lat, lon)
simulation_dt = timedelta(minutes=5)
save_dir = create_path("particledata/testcompare")

def DeleteParticle(particle, fieldset, time):
    particle.delete()

In [ ]:
for f in files:
    f["pset"] = ParticleSet(fieldset=f["fs"], pclass=JITParticle, lon=[spawn_loc[1]], lat=[spawn_loc[0]], time=[spawn_time])
    save_path = f"{save_dir}/{f['name']}.nc"
    f["pfile"] = f["pset"].ParticleFile(save_path, outputdt=timedelta(hours=1))
    
    f["pset"].execute(
                AdvectionRK4,
                endtime=end_time,
                dt=simulation_dt,
                recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle},
                output_file=f["pfile"]
            )
    f["pfile"].export()
    f["pfile"].close()

In [ ]:
p_ds = xr.open_dataset(files[0]["pfile"].name)
p_ds

In [ ]:
ps = [f["pfile"].name for f in files]
plot_trajectories(ps, files[0]["domain"])